In [13]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
tfk = tf.keras
tf.keras.backend.set_floatx("float64")
import tensorflow_probability as tfp
tfd = tfp.distributions
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
#setting up helper fucntions
scaler = StandardScaler()
detector = IsolationForest(n_estimators=1000, behaviour="deprecated", contamination="auto", random_state=0)
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)
#storing the data in a variable
data = pd.read_csv('../input/clothing-dataset-full/images.csv')
data.head()


In [33]:
data = data[['image','label']].groupby('label').count() #error at times but works fine the other times, why?

In [18]:
data = data[['image','label']].groupby('label').count()

In [16]:
category_cnt = data[['image','label']].groupby('label').count() #error at times but works at other times, why is that?
category_cnt.plot.bar() #error at times but works at other times, why is that?

In [ ]:
# training setup
inputs = ["image", "label"]
n_epochs = 35
n_samples = data.shape[0]
n_batches = 8
batch_size = np.floor(n_samples/8)
buffer_size = data.shape[0]

n_train = int(0.7*data.shape[0])
#training Bayesian Neural Network
#cutting/partitioning data
data = tf.data.Dataset.from_tensor_slices((data[inputs].values, data[inputs].values))
data = data.shuffle(n_samples, reshuffle_each_iteration=True)
#training data and taking respective data
data_train = data.take(n_train).batch(batch_size).repeat(n_epochs)
data_test = data.skip(n_train).batch(1)




In [ ]:
#Starting preprocessing/training
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(len(inputs), dtype=tf.float64), scale=1.0), reinterpreted_batch_ndims=1)

model = tfk.Sequential([
tfk.layers.InputLayer(input_shape=(len(inputs),), name="input"),
tfk.layers.Dense(10, activation="relu", name="dense_1"),
tfk.layers.Dense(tfp.layers.MultivariateNormalTriL.params_size(
len(inputs)), activation=None, name="distribution_weights"),
tfp.layers.MultivariateNormalTriL(len(inputs), activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/n_batches), name="inputs")
], name="model")
model.compile(optimizer="adam", loss=neg_log_likelihood)
#running training session
model.fit(data_train, epochs=n_epochs, validation_data=data_test, verbose=False)
model.summary()



# Define prior for regularization.
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(len(outputs), dtype=tf.float64), scale=1.0), reinterpreted_batch_ndims=1)
# Define model instance.
model = tfk.Sequential([
tfk.layers.InputLayer(input_shape=(len(inputs),), name="input"),
tfk.layers.Dense(10, activation="relu", name="dense_1"),
tfk.layers.Dense(tfp.layers.MultivariateNormalTriL.params_size(
len(outputs)), activation=None, name="distribution_weights"),
tfp.layers.MultivariateNormalTriL(len(outputs), activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/n_batches), name="output")
], name="model")
# Compile model.
model.compile(optimizer="adam", loss=neg_log_likelihood)
# Run training session.
model.fit(data_train, epochs=n_epochs, validation_data=data_test, verbose=False)
# Describe model.
model.summary()


In [ ]:
tfp.layers.DenseFlipout(10, activation="relu", name="dense_1")

In [ ]:
model.fit(data_train, epochs=n_epochs, validation_data=data_test, verbose=False)

# starting to predict data
samples = 200
iterations = 15
test_iterator = tf.compat.v1.data.make_one_shot_iterator(data_test)
X_true, Y_true, Y_pred = np.empty(shape=(samples, len(inputs))), np.empty(shape=(samples, len(inputs))), np.empty(shape=(samples, len(inputs), iterations))
for i in range(samples):
    features, labels = test_iterator.get_next()
    X_true[i,:] = features
    Y_true[i,:] = labels.numpy()
    for k in range(iterations):
        Y_pred[i,:,k] = model.predict(features)
        
# Calculating mean and standard deviation 
Y_pred_m = np.mean(Y_pred, axis=-1)
Y_pred_s = np.std(Y_pred, axis=-1)